## Use embeddings from OPENAI to embed any document of your choice.
### Create atleast 3 different docs by splitting one document

### Splitting documents is provided by text_splitter of langchain also, however As per my understanding,I am required to split the document into 3 different docs, so I have proceeeded that way.

### 1. Loading the document

In [2]:
from langchain.document_loaders import TextLoader

In [3]:
loader = TextLoader(r'./Data/Krishna.txt',encoding='utf8')
docs = loader.load()

### 2. Splitting the document into 3 or more documents

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

chunk_size = 100

text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=10)
texts = text_splitter.split_documents(docs)

In [5]:
n = len(texts)
while not (n>=3 and n<6):
    chunk_size += 100
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=10)
    texts = text_splitter.split_documents(docs)
    n = len(texts)
n

5

In [6]:
texts[0]

Document(page_content='Introduction\n\nLord Krishna is the 8th incarnation of Vishnu and the God of peace and\nlove. Many people worship Krishna including my family. Since I was young, I\nheard many stories of Krishna. I had always loved them and wanted to know more\nabout them.\nSince I worshipped Krishna, I was interested in what he had done and how\nKrishna had done them. Krishna had done lots of things like he fought off demons\nin his childhood, killed Kamsa, a wicked ruler, accompanied the Pandavas in their\nexile, and gave lessons to Arjuna, a Panadava before a war.\nI wondered, how was Krishna able to overcome all the things thrown at him?\nHow did Krishna change the war at that time? What were the meanings behind all\nof his stories? When Expert came around, I decided I would choose Lord Krishna\nbecause I was already interested and wanted to know more.\nEven though the teachings of Krishna are thousands of years old, Krishna’s\nteachings about peace are still relevant to peop

In [7]:
k= 1
for i in range(0,n,2):
        for j in range(i,i+2):
            with open(f'./Data/Files/Krishna{k}.txt', 'a') as fp:
                if j<n:
                    for item in (texts[j].page_content).split('\n'):
                        fp.write("%s\n" % item)
        k += 1
    
print('Done') 

Done


### 3. Loading again the 3 different documents created from the original document

In [8]:
from langchain.document_loaders import DirectoryLoader

In [9]:
loader = DirectoryLoader('./Data/Files',glob='**/*.txt')
docs = loader.load()

### 4. Split the documents into chunks

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=10)
texts = text_splitter.split_documents(docs)

### 5. Passing Open AI API Key

In [12]:
import os
import getpass

api_key = getpass.getpass('OpenAI API Key:')

OpenAI API Key: ········


### 6. Selecting the type of embedding to use

In [13]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(openai_api_key=api_key,model="text-embedding-ada-002")

### 7. Creating the vectorstore to use as the index

In [14]:
from langchain.vectorstores import Chroma
db = Chroma.from_documents(texts, embeddings)

In [15]:
retriever = db.as_retriever()

### 8. Creating a chain and use it to answer questions

In [16]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

In [17]:
qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=retriever,return_source_documents=True)

### 9. In-use Querying/Searching

In [19]:
question = "Where the town was located?"

In [20]:
output = qa(question)

In [21]:
output

{'query': 'Where the town was located?',
 'result': ' Gokula.',
 'source_documents': [Document(page_content='Introduction', metadata={'source': 'Data\\Files\\Krishna1.txt'}),
  Document(page_content='Vishnu. Vishnu turned to Vasudeva and gave him orders. He said to take Krishna into a town in Gokula. He told him where the town was located and how to get to it. Vishnu continued and said that a newborn girl resided in that village. Yashoda, the mother of the baby girl, was asleep. Go there and switch Krishna and the girl. Then bring the baby girl back here to the dungeon. Vasudeva and Devaki again thanked Vishnu and then he left. Vasudeva’s chains magically came undone, and he rose to carry out the orders he had been given by Vishnu. Vasudeva put Krishna in a basket and covered him in a blanket. He did not know how a prisoner could get out of prison without permission, but a blue light flashed through the dungeon that left the gates open and guards asleep. Vasudeva walked out of the dung

In [22]:
output['result']

' Gokula.'

In [23]:
question = input('Enter your question: ')
output = qa(question)
output['result']

Enter your question:  Who was Arjuna?


" Arjuna was a Pandava, a character in Hindu mythology, and one of Krishna's closest friends. Krishna gave lessons to Arjuna before a war."

In [24]:
output['source_documents']

[Document(page_content='Lord Krishna is the 8th incarnation of Vishnu and the God of peace and love. Many people worship Krishna including my family. Since I was young, I heard many stories of Krishna. I had always loved them and wanted to know more about them. Since I worshipped Krishna, I was interested in what he had done and how Krishna had done them. Krishna had done lots of things like he fought off demons in his childhood, killed Kamsa, a wicked ruler, accompanied the Pandavas in their exile, and gave lessons to Arjuna, a Panadava before a war. I wondered, how was Krishna able to overcome all the things thrown at him? How did Krishna change the war at that time? What were the meanings behind all of his stories? When Expert came around, I decided I would choose Lord Krishna because I was already interested and wanted to know more. Even though the teachings of Krishna are thousands of years old, Krishna’s teachings about peace are still relevant to people in the 21st century. Kris